Experiment with OpenCV for finding the pool table edges within an image.

This version assumes a full-sized vertical image of the table. It is looking
for the marker pattern along the side of the table.

In [ ]:
import cv2 as cv
import numpy as np
import numpy.linalg as linalg
import matplotlib.pyplot as plt
import math
import copy
import os
import importlib.util
from Utils import *

In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
files, dir = FindImageFilesAndDir(subdir="AmcrestCamera/Session2/checkerboards")
print("file count={0}".format(len(files)))
for fname in files:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)

In [ ]:

files, dir = FindImageFilesAndDir(subdir="AmcrestCamera/Session2/checkerboards")
print("file count={0}".format(len(files)))
images = []
patternSize = (9,6)

# Sub-Pixel setup
winSize = (5, 5)
zeroZone = (-1, -1)
criteria = (cv.TERM_CRITERIA_EPS + cv.TermCriteria_COUNT, 30, 0.001)

# Go find the pattern in each image
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
for file in files:
    image = cv.imread(file)
    image = cv.rotate(image,cv.ROTATE_90_CLOCKWISE)
    imageGray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    bFound, cornerPoints = cv.findChessboardCorners(imageGray, patternSize)
    if not bFound:
        continue
    objpoints.append(objp) # All the same
    cornerPointsSubPix = cv.cornerSubPix(imageGray, cornerPoints, winSize, zeroZone, criteria)
    imgpoints.append(cornerPoints)
    cv.drawChessboardCorners(image, patternSize, cornerPointsSubPix, bFound)
    images.append(image)
        
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, imageGray.shape[::-1], None, None)
    
img = images[0]
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))


# undistort and crop the image
imgUndistort = cv.undistort(img, mtx, dist, None, newcameramtx)
x, y, w, h = roi
#imgUndistort = imgUndistort[y:y+h, x:x+w]


ImgShow([img, imgUndistort], 200)


# Prototype moving to external functions

# Globals to hold the camera calibration

def GenerateCalibration(dirCheckerboard):
    files, dir = FindImageFilesAndDir(subdir=dirCheckerboard)
    print("file count={0}".format(len(files)))
    patternSize = (9,6)

    # Sub-Pixel setup
    winSize = (5, 5)
    zeroZone = (-1, -1)
    criteria = (cv.TERM_CRITERIA_EPS + cv.TermCriteria_COUNT, 30, 0.001)

    # Go find the pattern in each image
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    for file in files:
        image = cv.imread(file)
        image = cv.rotate(image,cv.ROTATE_90_CLOCKWISE)
        imageGray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        bFound, cornerPoints = cv.findChessboardCorners(imageGray, patternSize)
        if not bFound:
            continue
        objpoints.append(objp) # All the same
        cornerPointsSubPix = cv.cornerSubPix(imageGray, cornerPoints, winSize, zeroZone, criteria)
        imgpoints.append(cornerPoints)
        cv.drawChessboardCorners(image, patternSize, cornerPointsSubPix, bFound)
        images.append(image)
        
    global calMatrix  
    ret, calMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, imageGray.shape[::-1], None, None)
        
    img = images[0]
    h, w = img.shape[:2]
        
    global calMatrixOptimal
    global calROI
    calMatrixOptimal, calROI = cv.getOptimalNewCameraMatrix(calMatrix, dist, (w,h), 1, (w,h))
    

def CalibrateImage(img):    
    # undistort and crop the image
    global calMatrix  
    global calMatrixOptimal
    imgUndistort = cv.undistort(img, calMatrix, dist, None, calMatrixOptimal)
    x, y, w, h = calROI
    #imgUndistort = imgUndistort[y:y+h, x:x+w]
    return imgUndistort

imgA = images[1]
GenerateCalibration("AmcrestCamera/Session2/checkerboards")
imgB = CalibrateImage(imgA)
ImgShow([imgA, imgB], 200)